# Pronostico de potencia

In [1]:
import datetime
import inspect
import os

# scientific python add-ons
import numpy as np
import pandas as pd

# plotting stuff
# first line makes the plots appear in the notebook
%matplotlib inline 
import matplotlib.pyplot as plt
import matplotlib as mpl

# finally, we import the pvlib library
import pvlib
from pvlib import solarposition, irradiance, atmosphere, pvsystem, inverter, temperature, iam
from pvlib.forecast import GFS, NAM, NDFD, RAP, HRRR

/Users/luisvargas/Desktop/Tesis Pvlib/env_pvlib_tesis/lib/python3.8/site-packages/pvlib/forecast.py:18: UserWarning: The forecast module algorithms and features are highly experimental. The API may change, the functionality may be consolidated into an io module, or the module may be separated into its own package.
  warnings.warn(


### Load Forecast data

pvlib forecast module only includes several models. To see the full list of forecast models visit the Unidata website:

http://www.unidata.ucar.edu/data/#tds


In [2]:
# Choose a location.
# Tucson, AZ
latitude = 20.56
longitude = -103.22 
tz = 'America/Mexico_City'

Definir algunos parametros del sistema fotovoltaico.

In [3]:
surface_tilt = 30
surface_azimuth = 180 # pvlib uses 0=North, 90=East, 180=South, 270=West convention
albedo = 0.2

In [4]:
start = pd.Timestamp(datetime.date.today(), tz=tz) # today's date
end = start + pd.Timedelta(days=7) # 7 days from today

In [5]:
# Define forecast model
fm = GFS()
#fm = NAM()
#fm = NDFD()
#fm = RAP()
#fm = HRRR()

In [6]:
# Retrieve data
forecast_data = fm.get_processed_data(latitude, longitude, start, end)

In [7]:
forecast_data.head()

,temp_air,wind_speed,ghi,dni,dhi,total_clouds,low_clouds,mid_clouds,high_clouds
2020-11-26 06:00:00-06:00,14.149994,1.148972,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2020-11-26 09:00:00-06:00,12.845245,0.427756,325.053195,642.323957,85.142313,0.0,0.0,0.0,0.0
2020-11-26 12:00:00-06:00,12.427887,1.444086,653.846474,524.910637,268.914159,20.0,0.0,0.0,20.0
2020-11-26 15:00:00-06:00,22.250427,0.927074,519.269418,563.361710,187.110158,16.0,0.0,0.0,16.0
2020-11-26 18:00:00-06:00,38.149994,0.629852,2.777806,0.000000,2.777806,8.0,0.0,0.0,8.0


In [8]:
ghi = forecast_data['ghi']
#ghi.plot()
#plt.ylabel('Irradiance ($W/m^{-2}$)');


### Calculate modeling intermediates

Before we can calculate power for all the forecast times, we will need to calculate:

    solar position
    extra terrestrial radiation
    airmass
    angle of incidence
    POA sky and ground diffuse radiation
    cell and module temperatures

The approach here follows that of the pvlib tmy_to_power notebook. You will find more details regarding this approach and the values being calculated in that notebook.
Solar position

Calculate the solar position for all times in the forecast data.

The default solar position algorithm is based on Reda and Andreas (2004). Our implementation is pretty fast, but you can make it even faster if you install numba and use add method='nrel_numba' to the function call below.


In [9]:
# retrieve time and location parameters
time = forecast_data.index
a_point = fm.location

In [10]:
solpos = a_point.get_solarposition(time)
#solpos.plot()

In [11]:
dni_extra = irradiance.get_extra_radiation(fm.time)
#dni_extra.plot()
#plt.ylabel('Extra terrestrial radiation ($W/m^{-2}$)')

In [12]:
airmass = atmosphere.get_relative_airmass(solpos['apparent_zenith'])
#airmass.plot()
#plt.ylabel('Airmass')

In [13]:
poa_sky_diffuse = irradiance.haydavies(surface_tilt, surface_azimuth,
                                       forecast_data['dhi'], forecast_data['dni'], dni_extra,
                                       solpos['apparent_zenith'], solpos['azimuth'])
#poa_sky_diffuse.plot()
#plt.ylabel('Irradiance ($W/m^{-2}$)')

In [14]:
poa_ground_diffuse = irradiance.get_ground_diffuse(surface_tilt, ghi, albedo=albedo)
#poa_ground_diffuse.plot()
#plt.ylabel('Irradiance ($W/m^{-2}$)')

In [15]:
aoi = irradiance.aoi(surface_tilt, surface_azimuth, solpos['apparent_zenith'], solpos['azimuth'])
#aoi.plot()
#plt.ylabel('Angle of incidence (deg)')

In [16]:
poa_irrad = irradiance.poa_components(aoi, forecast_data['dni'], poa_sky_diffuse, poa_ground_diffuse)
#poa_irrad.plot()
#plt.ylabel('Irradiance ($W/m^{-2}$)')
#plt.title('POA Irradiance');

In [28]:
poa_direct = poa_irrad['poa_direct']

In [30]:
poa_diffuse = poa_irrad['poa_diffuse']

In [33]:
poa_global = poa_irrad['poa_global']

In [17]:
ambient_temperature = forecast_data['temp_air']
wnd_spd = forecast_data['wind_speed']
thermal_params = temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer']
pvtemp = temperature.sapm_cell(poa_irrad['poa_global'], ambient_temperature, wnd_spd, **thermal_params)
#pvtemp.plot()
#plt.ylabel('Temperature (C)');

In [18]:
cec_modules = pvsystem.retrieve_sam('cecmod')

In [19]:
cec_module = cec_modules.Canadian_Solar_Inc__CS6X_320P

In [20]:
sandia_modules = pvsystem.retrieve_sam('SandiaMod')

In [21]:
sandia_module = sandia_modules.Canadian_Solar_CS5P_220M___2009_

In [22]:
#effective_irradiance = pvsystem.sapm_effective_irradiance(poa_irrad.poa_direct, poa_irrad.poa_diffuse,airmass, aoi, cec_module)
iam = iam.martin_ruiz(aoi)    # or choose a different IAM model from pvlib.iam

In [31]:
effective_irradiance = poa_direct * iam + poa_diffuse
effective_irradiance

2020-11-26 06:00:00-06:00      0.000000
2020-11-26 09:00:00-06:00    476.090729
2020-11-26 12:00:00-06:00    804.487118
2020-11-26 15:00:00-06:00    671.704074
2020-11-26 18:00:00-06:00      2.628944
2020-11-26 21:00:00-06:00      0.000000
2020-11-27 00:00:00-06:00      0.000000
2020-11-27 03:00:00-06:00      0.000000
2020-11-27 06:00:00-06:00      0.000000
2020-11-27 09:00:00-06:00    232.778165
2020-11-27 12:00:00-06:00    720.713382
2020-11-27 15:00:00-06:00    397.369338
2020-11-27 18:00:00-06:00      2.276272
2020-11-27 21:00:00-06:00      0.000000
2020-11-28 00:00:00-06:00      0.000000
2020-11-28 03:00:00-06:00      0.000000
2020-11-28 06:00:00-06:00      0.000000
2020-11-28 09:00:00-06:00    206.624874
2020-11-28 12:00:00-06:00    575.359684
2020-11-28 15:00:00-06:00    785.227884
2020-11-28 18:00:00-06:00      2.834867
2020-11-28 21:00:00-06:00      0.000000
2020-11-29 00:00:00-06:00      0.000000
2020-11-29 03:00:00-06:00      0.000000
2020-11-29 06:00:00-06:00      0.000000


In [32]:
sapm_out = pvsystem.sapm(effective_irradiance, pvtemp, cec_module)
#print(sapm_out.head())
sapm_out[['p_mp']].plot()
plt.ylabel('DC Power (W)');

KeyError: 'Bvmpo'

In [ ]:
sapm_inverters = pvsystem.retrieve_sam('sandiainverter')

In [ ]:
sapm_inverter = sapm_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']
sapm_inverter

In [ ]:
p_ac = inverter.sandia(sapm_out.v_mp, sapm_out.p_mp, sapm_inverter)

p_ac.plot()
plt.ylabel('AC Power (W)')
plt.ylim(0, None);

In [ ]:
p_ac[start:start+pd.Timedelta(days=2)].plot();

In [ ]:
p_ac.describe()

In [ ]:
p_ac.index.freq

In [ ]:
# integrate power to find energy yield over the forecast period
p_ac.sum() * 3